#### -----------------------------------------------------------------------------<br>Copyright (c) 2022, Lucid Vision Labs, Inc.
##### THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN  THE  SOFTWARE.<br>-----------------------------------------------------------------------------

In [ ]:
import time
from arena_api.system import system

#### Acquisition: Sensor Binning
>    This example demonstrates how to configure device settings to enable binning
    at the sensor level, so that the sensor will combine rectangles of pixels into
    larger "bins". This results in reduced resolution of images, but also reduces
    the amount of data sent to the software and networking layers.

In [ ]:
"""
This function waits for the user to connect a device before raising
an exception
"""

tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
        tries += 1
    else:
        print(f'Created {len(devices)} device(s)')
        device = devices[0]
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')


##### Check if sensor binning is supported

In [ ]:
nodemap = device.nodemap
binning_selector = nodemap.get_node("BinningSelector")

print("Checking if sensor binning is supported")
if("Sensor" not in binning_selector.enumentry_names or
        not binning_selector.enumentry_nodes.get("Sensor").is_readable):
    print("Sensor binning not supported by device: not available from BinningSelector")
    quit()


##### Configure devices and store initial values
> These initial values are restored to the device after the example is completed

In [ ]:
acquisitionModeInitial = nodemap.get_node("AcquisitionMode").value
binningSelectorInitial = nodemap.get_node("BinningSelector").value

binningVerticalModeInitial = nodemap.get_node("BinningHorizontalMode").value
binningHorizontalModeInitial = nodemap.get_node("BinningHorizontalMode").value

binningVerticalInitial = nodemap.get_node("BinningVertical").value
binningHorizontalInitial = nodemap.get_node("BinningHorizontal").value

##### Activate sensor binning and maximize bin size

##### This is the entry we will use for BinningVerticalMode and BinningHorizontalMode. 
>    Sum will result in a brighter image, compared to Average.

In [ ]:
BINTYPE = "Sum"

binning_selector.value = "Sensor"

##### Set BinningHorizontal and BinningVertical to their maxes.
>    This sets width and height of the bins: the number of pixels along each axis.
    Maximizes compression

In [ ]:
binning_vertical_node = nodemap.get_node("BinningVertical")
binning_horizontal_node = nodemap.get_node("BinningHorizontal")

if (not nodemap.get_node("BinningVertical").is_writable or
        not nodemap.get_node("BinningVertical").is_writable):
    print("Sensor binning is not supported: "
          "BinningVertical or BinningHorizontal not available")
    quit()

In [ ]:
print("Find max bin height and width")
bin_height = binning_vertical_node.max
bin_width = binning_horizontal_node.max

##### Set BinningHorizontal and BinningVertical to their maxes.
>    This sets width and height of the bins: the number of pixels along each axis.

In [ ]:
print(f"Set bin height and width to {bin_height} and {bin_width} respectively")
binning_vertical_node.value = bin_height
binning_horizontal_node.value = bin_width

##### Sets binning mode
>    Sets binning mode for horizontal and vertical axes.
    Generally, they're set to the same value.

In [ ]:
print(f"Set binning mode to {BINTYPE}")
nodemap.get_node("BinningVerticalMode").value = BINTYPE
nodemap.get_node("BinningHorizontalMode").value = BINTYPE

##### Setup stream values

In [ ]:

tl_stream_nodemap = device.tl_stream_nodemap

nodemap.get_node("AcquisitionMode").value = "Continuous"

# enable stream auto negotiate packet size
tl_stream_nodemap['StreamAutoNegotiatePacketSize'].value = True

# enable stream packet resend
tl_stream_nodemap['StreamPacketResendEnable'].value = True


##### Grab images -------------------------------------------------------------
> - Starting the stream allocates buffers, which can be passed in as
 an argument (default: 10), and begins filling them with data.
 Buffers must later be requeued to avoid memory leaks.<br>
> - 'Device.get_buffer()' with no arguments returns one buffer(NOT IN A LIST)<br>
> - 'Device.get_buffer(30)' returns 30 buffers(IN A LIST)<br>
> - 'Device.requeue_buffer()' takes a buffer or many buffers in a list or tuple

In [ ]:
number_of_buffers = 30

device.start_stream(number_of_buffers)
print(f'Stream started with {number_of_buffers} buffers')

print(f'\tGet {number_of_buffers} buffers in a list')
buffers = device.get_buffer(number_of_buffers)

# Print image buffer info
for count, buffer in enumerate(buffers):
    print(f'\t\tbuffer{count:{2}} received | '
          f'Width = {buffer.width} pxl, '
          f'Height = {buffer.height} pxl, '
          f'Pixel Format = {buffer.pixel_format.name}')

device.requeue_buffer(buffers)
print(f'Requeued {number_of_buffers} buffers')

device.stop_stream()
print(f'Stream stopped')

##### Clean up ----------------------------------------------------------------
> - Restore initial values to the device.
> - Destroy device. This call is optional and will automatically be
  called for any remaining devices when the system module is unloading.

In [ ]:
print("Restoring initial configurations")
nodemap.get_node("BinningVertical").value = binningVerticalInitial
nodemap.get_node("BinningHorizontal").value = binningHorizontalInitial

nodemap.get_node("BinningHorizontalMode").value = binningVerticalModeInitial
nodemap.get_node("BinningHorizontalMode").value = binningHorizontalModeInitial

nodemap.get_node("BinningSelector").value = binningSelectorInitial 
nodemap.get_node("AcquisitionMode").value = acquisitionModeInitial

In [ ]:
system.destroy_device()
print("Destroyed all created devices")